# FP - TRUN

In [2]:
from datetime import datetime
import mRun as mr
import utils_data as md
import numpy as np
import pandas as pd

def get_models(type):
    if type == "FRFLO":
        return [
            { 'dt':'C2',  "e":100, "lr":0.001, "h":[100 , 100], "spn": 5000, "pe": [], "pt": []  },
            { 'dt':'C4',  "e":100, "lr":0.001, "h":[100 , 100], "spn": 5000, "pe": [], "pt": []  },
            { 'dt':'C1',  "e":100, "lr":0.001, "h":[100 , 100], "spn": 5000, "pe": [], "pt": []  },
        ]
    elif type == "FRALL1":
        return [
            { 'dt':'C2',  "e":40,  "lr":0.001, "h":[100 , 100], "spn": 40000, "pe": [], "pt": []  },
            # { 'dt':'C4',  "e":100, "lr":0.001, "h":[100 , 100], "spn": 40000, "pe": [], "pt": []  },
            # { 'dt':'C1',  "e":100, "lr":0.001, "h":[100 , 100], "spn": 40000, "pe": [], "pt": []  },
            # { 'dt':'C0',  "e":100, "lr":0.001, "h":[100 , 100], "spn": 10000, "pe": [], "pt": []  },
        ]
    else: return []

In [4]:
print("___Start!___" +  datetime.now().strftime('%H:%M:%S')  )
final = "_" ;  md.DESC = "FRALL1";  # FRFLO   FRALL1
ALL_DS = md.LOGDAT + md.DESC + md.DSC 

execc = get_models(md.DESC)

# DATA READ  ------------------------------------------------ 
# md.mainRead2(ALL_DS, 1, 2 ) # , all = True, shuffle = True  ) 
md.mainRead2(path=ALL_DS, part=1, batch_size=2 ) # For testing I am forced to used JSON - column names and order may be different! 

url_test = md.LOGDAT + "FREXP1/" ; # url_test = "url"
force = False; excel = True  # dataFile = "frall2_json.txt"; labelFile = "datal.csv"     
md.get_tests(url_test, force, excel )


___Start!___01:04:31
data read - 45822 - time:49.66408729553223


In [5]:
# OPERATIONS  ------------------------------------------------ 
# md.get_columns(force)
for ex in execc:
    md.spn = ex["spn"]; md.dType = ex["dt"]; mr.epochs = ex["e"]; mr.lr = ex["lr"]; mr.h = ex["h"]

    md.normalize()  
    mr.ninp, mr.nout, mr.top_k = md.getnn()
    md.MODEL_DIR = md.LOGDIR + md.DESC + '/'   + mr.get_hpar(mr.epochs, final=final) +"/" 
    mr.model_path = md.MODEL_DIR + "model.ckpt" 
    mr.build_network3()                                                                                                                                                                                                                                                                                    
    print(mr.model_path)    
    # ex["pe"] = mr.evaluate( )
    ex["pt"] = mr.tests(url_test, p_col=False  )

# PRINTING  ------------------------------------------------ 
print("end!___" +  datetime.now().strftime('%H:%M:%S')  )

build network
../../dmodels/FRALL1/slr_1E-03_NN2385x100x100x2_ep40_/model.ckpt
_____TESTS...
Model restored from file: ../../dmodels/FRALL1/slr_1E-03_NN2385x100x100x2_ep40_/model.ckpt
INFO:tensorflow:Restoring parameters from ../../dmodels/FRALL1/slr_1E-03_NN2385x100x100x2_ep40_/model.ckpt
test ac = 0.99335235
0 RealVal: 1 - 1 - PP: [1 0] PR: [9.999914e-01 8.533054e-06]
1 RealVal: 1 - 1 - PP: [1 0] PR: [1.0000000e+00 5.5183982e-08]
2 RealVal: 1 - 1 - PP: [1 0] PR: [9.999907e-01 9.355504e-06]
3 RealVal: 1 - 1 - PP: [1 0] PR: [9.999988e-01 1.168963e-06]
4 RealVal: 1 - 1 - PP: [1 0] PR: [1.0000000e+00 2.6773737e-08]
5 RealVal: 1 - 1 - PP: [1 0] PR: [9.9997807e-01 2.1874701e-05]
6 RealVal: 1 - 1 - PP: [1 0] PR: [9.9996257e-01 3.7446203e-05]
7 RealVal: 1 - 1 - PP: [1 0] PR: [1.0000000e+00 4.0664194e-16]
8 RealVal: 1 - 1 - PP: [1 0] PR: [9.999999e-01 8.128393e-08]
9 RealVal: 1 - 1 - PP: [1 0] PR: [9.9999952e-01 4.4085803e-07]
10 RealVal: 1 - 1 - PP: [1 0] PR: [9.9999905e-01 9.8073474e-07]
11

In [7]:
#save copy
# md.normalize()  = apply inverse normalization!
tsd = md.dsp[["M", "FP"]]
tsd.insert(len(tsd.columns), ex["dt"] + 'FP_P', md.dsp["FP_P"].map(lambda x: md.dc( x ) ))    

# for ex in execc:
def record_d( ex, dsp, type = "pt"  ):

    print(len(np.array([str(xi[0]) for xi in ex["pt"][1]])))
    print(len(tsd.columns))
    
    tsd.insert(len(tsd.columns), ex["dt"] + 'FP_P', dsp["FP_P"].map(lambda x: dc( x ) )    )
    
        
    #tsd[ "_PRED"] = np.array([str(xi[0]) for xi in ex["pt"][1]])  
    #tsd.insert(len(tsd.columns), ex["dt"] + '_PRED', np.array([str(xi[0]) for xi in ex["pt"][1]])  )    
    tsd.insert(len(tsd.columns), ex["dt"] + '_PREDU', np.array([str(xi[0]) for xi in ex["pt"][1]])  )    
    tsd.insert(len(tsd.columns), ex["dt"] + '_PRED', np.array([str(xi) for xi in ex["pt"][1]])  )    
    tsd.insert(len(tsd.columns), ex["dt"] + '_PROB', np.array([str(xi) for xi in ex["pt"][0]])  )    

    # error calc: 
    #tsd[ex["dt"] + '_ERR'] = tsd.[[ex["dt"] + '_PREDU']] == 
    
    # tsd["PRED"] = ex["pt"][1][i][0] # pred 1
    # tsd[ex["dt"] + "_PRED"] = ex["pt"][1] # pred 1
    # tsd[ex["dt"] + "_PROB"] = ex["pt"][0] # prob 

    #tsd.insert(2, '_PRED', np.array([str(xi[0]) for xi in ex["pt"][1]])  )    
    #tsd[ex["dt"] + "_PROB"] = ex["pt"][0] # prob 
    
#tsd.to_csv(md.LOGDAT + "testDSJ.csv")
tsd.head()
# md.dsp.head()


,M,FP,C2FP_P
0,640309,70,1
1,640310,74,1
2,640391,67,1
3,640411,67,1
4,PAV10150AP,97,1
